Presentation Link: https://hcpss-my.sharepoint.com/:p:/g/personal/tcarte6482_inst_hcpss_org/EaypnPAeNSlAupVA95ZEl64BHHoIF1MtkDF6yXVWWNcysQ?e=NueUfT

#Visualizing the Data

In [ ]:
!pip install utm
!pip install geopy
!pip install pcl
!python --version
!pip install open3d
!pip install pyntcloud
#!pip install pcl.pcl_visualization



In [ ]:
import matplotlib.pyplot as plt

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import sys
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import ConfusionMatrixDisplay
import pandas as pd
import math
import utm
import geopy.distance
from geopy.distance import geodesic
import cv2
import yaml
import json
from scipy.spatial.transform import Rotation as R
import matplotlib.patches as patches

#import pcl
#import pcl.pcl_visualization
import json

from sklearn.neighbors import KDTree
import open3d as o3d
from pyntcloud import PyntCloud
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_route_stats = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cadc_dataset_route_stats.csv')

In [ ]:
dataset_route_stats.head(75)
#dataset_route_stats['Dataset Type'].value_counts()

In [ ]:
def convert_novatel_to_pose(novatel):
  poses = [];
  FIRST_RUN = True;
  origin = [];

  for gps_msg in novatel:
    # utm_data[0] = East (m), utm_data[1] = North (m)
    utm_data = utm.from_latlon(float(gps_msg[0]), float(gps_msg[1]));
    # Ellipsoidal height = MSL (orthometric) + undulation
    ellipsoidal_height = float(gps_msg[2]) + float(gps_msg[3]);

    roll = np.deg2rad(float(gps_msg[7]));
    pitch = np.deg2rad(float(gps_msg[8]));

    # Azimuth = north at 0 degrees, east at 90 degrees, south at 180 degrees and west at 270 degrees
    azimuth = float(gps_msg[9]);
    # yaw = north at 0 deg, 90 at west and 180 at south, east at 270 deg
    yaw = np.deg2rad(-1.0 * azimuth);

    c_phi = math.cos(roll);
    s_phi = math.sin(roll);
    c_theta = math.cos(pitch);
    s_theta = math.sin(pitch);
    c_psi = math.cos(yaw);
    s_psi = math.sin(yaw);

    if FIRST_RUN:
      origin = [utm_data[0], utm_data[1], ellipsoidal_height];
      FIRST_RUN = False;

    # This is the T_locallevel_body transform where ENU is the local level frame
    # and the imu is the body frame
    # https://www.novatel.com/assets/Documents/Bulletins/apn037.pdf
    poses.append(np.matrix([
      [c_psi * c_phi - s_psi * s_theta * s_phi, -s_psi * c_theta, c_psi * s_phi + s_psi * s_theta * c_phi, utm_data[0] - origin[0]],
      [s_psi * c_phi + c_psi * s_theta * s_phi, c_psi * c_theta, s_psi * s_phi - c_psi * s_theta * c_phi, utm_data[1] - origin[1]],
      [-c_theta * s_phi, s_theta, c_theta * c_phi, ellipsoidal_height - origin[2]],
      [0.0, 0.0, 0.0, 1.0]]));

  return poses;


In [ ]:
def load_calibration(calib_path):
  calib = {}

  # Get calibrations
  calib['extrinsics'] = yaml.load(open(calib_path + '/extrinsics.yaml'), yaml.SafeLoader)
  calib['CAM00'] = yaml.load(open(calib_path + '/00.yaml'), yaml.SafeLoader)
  calib['CAM01'] = yaml.load(open(calib_path + '/01.yaml'), yaml.SafeLoader)
  calib['CAM02'] = yaml.load(open(calib_path + '/02.yaml'), yaml.SafeLoader)
  calib['CAM03'] = yaml.load(open(calib_path + '/03.yaml'), yaml.SafeLoader)
  calib['CAM04'] = yaml.load(open(calib_path + '/04.yaml'), yaml.SafeLoader)
  calib['CAM05'] = yaml.load(open(calib_path + '/05.yaml'), yaml.SafeLoader)
  calib['CAM06'] = yaml.load(open(calib_path + '/06.yaml'), yaml.SafeLoader)
  calib['CAM07'] = yaml.load(open(calib_path + '/07.yaml'), yaml.SafeLoader)

  return calib

In [ ]:
def load_novatel_data(novatel_path):
  files = os.listdir(novatel_path);
  novatel = [];

  for file in sorted(files):
    with open(novatel_path + file) as fp:
      novatel.append(fp.readline().split(' '));

  return novatel;

In [ ]:
class lidar_utils:
  def __init__(self, T_CAM_LIDAR):
    self.T_CAM_LIDAR = T_CAM_LIDAR;
    print("init lidar utils")

  def project_points(self, img, lidar_path, T_IMG_CAM, T_CAM_LIDAR, dist_coeffs, DISTORTED):
    self.T_CAM_LIDAR = T_CAM_LIDAR;

    scan_data = np.fromfile(lidar_path, dtype=np.float32);

    # 2D array where each row contains a point [x, y, z, intensity]
    lidar = scan_data.reshape((-1, 4));

    # Get height and width of the image
    h, w = img.shape[:2]

    projected_points = [];

    [rows, cols] = lidar.shape;
    # print(lidar[0,:])
    # print(lidar[1,:])
    # print(lidar[1,0:3])

    for i in range(rows):
      # print(lidar[i,:])
      p = np.array([0.0, 0.0, 0.0, 1.0]);
      p[0:3] = lidar[i,0:3];
      # print("p",p);
      projected_p =  np.matmul(self.T_CAM_LIDAR, p.transpose());
      if projected_p[2] < 2: # arbitrary cut off
        continue;
      projected_points.append([projected_p[0], projected_p[1], projected_p[2]]);

    #print("projected_points", projected_points)

    # Send [x, y, z] and Transform
    projected_points_np = np.array(projected_points)
    image_points = self.project(projected_points_np, T_IMG_CAM, dist_coeffs, DISTORTED);
    # print("image_points")
    # print(image_points)

    radius = 0

    [rows, cols] = projected_points_np.shape;

    NUM_COLOURS = 7;
    rainbow = [
      [0, 0, 255], # Red
      [0, 127, 255], # Orange
      [0, 255, 255], # Yellow
      [0, 255, 0], # Green
      [255, 0, 0], # Blue
      [130, 0, 75], # Indigo
      [211, 0, 148] # Violet
    ];

    for i in range(rows):
      colour = int(NUM_COLOURS*(projected_points_np[i][2]/70));
      x = int(image_points[i][0])
      y = int(image_points[i][1])
      if x < 0 or x > w - 1 or y < 0 or y > h - 1:
        continue;
      if colour > NUM_COLOURS-1:
        continue;

      cv2.circle(img, (x,y), radius, rainbow[colour], thickness=2, lineType=8, shift=0);

    return img;

  def project(self, p_in, T_IMG_CAM, dist_coeffs, DISTORTED):
    p_out = []
    [rows, cols] = p_in.shape;

    for i in range(rows):
      # print("p_in[i]", p_in[i])
      point = np.array([0.0, 0.0, 0.0, 1.0]);
      # print("p_in[i][0]",p_in[i][0])
      point[0:3] = p_in[i];
      # print("p[0]",p[0])
      if DISTORTED:
        rvec = tvec = np.zeros(3)
        # print(p[0:3])
        # print(T_IMG_CAM[0:3,0:3])
        # print(np.array(calib['CAM02']['distortion_coefficients']['data']))
        image_points, jac = cv2.projectPoints(np.array([point[0:3]]), rvec, tvec, T_IMG_CAM[0:3,0:3], dist_coeffs)
        p_out.append([image_points[0,0,0], image_points[0,0,1]]);
        # print("image_points", image_points[0,0])
      else:
        curr = np.matmul(T_IMG_CAM, point.transpose()).transpose();
        # print("curr",curr);
        done = [curr[0] / curr[2], curr[1] / curr[2]]
        p_out.append(done);
        # print("p_out append", done)

    return p_out;

In [ ]:
def run_demo_vehicle_path():
  #novatel_path = '/media/matthew/WAVELAB_2TB/winter/data/0027/processed/novatel/data/';
  novatel_path = '/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/0001/labeled/novatel/data/'
  novatel = load_novatel_data(novatel_path);
  poses = convert_novatel_to_pose(novatel);

  mpl.rcParams['legend.fontsize'] = 10

  fig = plt.figure()
  #ax = fig.gca(111, projection = '3d')
  ax = fig.add_subplot(111, projection='3d')


  ax.set_title('Vehicle path')
  ax.set_xlabel('East (m)')
  ax.set_ylabel('North (m)')
  ax.set_zlabel('Up (m)')

  length = 1
  A = np.matrix([[0, 0, 0, 1],
                [length, 0, 0, 1],
                [0, 0, 0, 1],
                [0, length, 0, 1],
                [0, 0, 0, 1],
                [0, 0, length, 1]]).transpose();

  for pose in poses:
    B = np.matmul(pose, A);
    ax.plot([B[0,0], B[0,1]], [B[1,0], B[1,1]],[B[2,0],B[2,1]], 'r-'); # x: red
    ax.plot([B[0,2], B[0,3]], [B[1,2], B[1,3]],[B[2,2],B[2,3]], 'g-'); # y: green
    ax.plot([B[0,4], B[0,5]], [B[1,4], B[1,5]],[B[2,4],B[2,5]], 'b-'); # z: blue

  # Equal axis doesn't seem to work so set an arbitrary limit to the z axis
  ax.set_zlim3d(-10,10)

  plt.show()
run_demo_vehicle_path()

In [ ]:
def run_demo_tracklets():
  frame = 26
  cam = '0'
  seq = '0010'
  DISTORTED = False
  MOVE_FORWARD = False
  # BASE = "/media/matthew/WAVELAB_2TB/winter/data/"
  #BASE = "/media/matthew/MOOSE-4TB/2019_02_27/"
  #CALIB_BASE = "/media/matthew/WAVELAB_2TB/winter/"
  BASE = "/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/"
  CALIB_BASE = "/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/"

  if DISTORTED:
    path_type = 'raw'
  else:
    path_type = 'processed'
  path_type = 'labeled'

  lidar_path = BASE + seq + "/" + path_type + "/lidar_points/data/" + format(frame, '010') + ".bin";
  calib_path = CALIB_BASE + "calib";
  img_path = BASE + seq + "/" + path_type + "/image_0" + cam + "/data/" + format(frame, '010') + ".png";



  annotations_file = BASE + seq + "/3d_ann.json";

  # Load 3d annotations
  annotations_data = None
  with open(annotations_file) as f:
      annotations_data = json.load(f)

  calib = load_calibration(calib_path);

  # Projection matrix from camera to image frame
  T_IMG_CAM = np.eye(4);
  T_IMG_CAM[0:3,0:3] = np.array(calib['CAM0' + cam]['camera_matrix']['data']).reshape(-1, 3);
  T_IMG_CAM = T_IMG_CAM[0:3,0:4]; # remove last row

  T_CAM_LIDAR = np.linalg.inv(np.array(calib['extrinsics']['T_LIDAR_CAM0' + cam]));

  T_IMG_LIDAR = np.matmul(T_IMG_CAM, T_CAM_LIDAR);

  img = cv2.imread(img_path)
  img_h, img_w = img.shape[:2]

  # Add each cuboid to image
  for cuboid in annotations_data[frame]['cuboids']:
    T_Lidar_Cuboid = np.eye(4);
    T_Lidar_Cuboid[0:3,0:3] = R.from_euler('z', cuboid['yaw'], degrees=False).as_matrix();
    T_Lidar_Cuboid[0][3] = cuboid['position']['x'];
    T_Lidar_Cuboid[1][3] = cuboid['position']['y'];
    T_Lidar_Cuboid[2][3] = cuboid['position']['z'];

    #T_Lidar_Cuboid[0][3] = -T_Lidar_Cuboid[0][3];

    # if (cuboid['label'] != 'Truck'):
    #   continue;
    # if (cuboid['attributes']['truck_type'] != 'Semi_Truck'):
    #   continue;
    # print(cuboid['yaw'])
    # print(cuboid)
    # print(T_Lidar_Cuboid)
    width = cuboid['dimensions']['x'];
    length = cuboid['dimensions']['y'];
    height = cuboid['dimensions']['z'];
    radius = 3

    # Create circle in middle of the cuboid
    tmp = np.matmul(T_CAM_LIDAR, T_Lidar_Cuboid);
    if tmp[2][3] < 0: # Behind camera
      continue;
    test = np.matmul(T_IMG_CAM, tmp);
    x = int(test[0][3]/test[2][3]);
    y = int(test[1][3]/test[2][3]);
    cv2.circle(img, (x,y), radius, [0, 0, 255], thickness=2, lineType=8, shift=0);

    front_right_bottom = np.array([[1,0,0,length/2],[0,1,0,-width/2],[0,0,1,-height/2],[0,0,0,1]]);
    front_right_top = np.array([[1,0,0,length/2],[0,1,0,-width/2],[0,0,1,height/2],[0,0,0,1]]);
    front_left_bottom = np.array([[1,0,0,length/2],[0,1,0,width/2],[0,0,1,-height/2],[0,0,0,1]]);
    front_left_top = np.array([[1,0,0,length/2],[0,1,0,width/2],[0,0,1,height/2],[0,0,0,1]]);

    back_right_bottom = np.array([[1,0,0,-length/2],[0,1,0,-width/2],[0,0,1,-height/2],[0,0,0,1]]);
    back_right_top = np.array([[1,0,0,-length/2],[0,1,0,-width/2],[0,0,1,height/2],[0,0,0,1]]);
    back_left_bottom = np.array([[1,0,0,-length/2],[0,1,0,width/2],[0,0,1,-height/2],[0,0,0,1]]);
    back_left_top = np.array([[1,0,0,-length/2],[0,1,0,width/2],[0,0,1,height/2],[0,0,0,1]]);

    # Project to image
    tmp = np.matmul(T_CAM_LIDAR, np.matmul(T_Lidar_Cuboid, front_right_bottom));
    if tmp[2][3] < 0:
      continue;
    f_r_b = np.matmul(T_IMG_CAM, tmp);
    tmp = np.matmul(T_CAM_LIDAR, np.matmul(T_Lidar_Cuboid, front_right_top));
    if tmp[2][3] < 0:
      continue;
    f_r_t = np.matmul(T_IMG_CAM, tmp);
    tmp = np.matmul(T_CAM_LIDAR, np.matmul(T_Lidar_Cuboid, front_left_bottom));
    if tmp[2][3] < 0:
      continue;
    f_l_b = np.matmul(T_IMG_CAM, tmp);
    tmp = np.matmul(T_CAM_LIDAR, np.matmul(T_Lidar_Cuboid, front_left_top));
    if tmp[2][3] < 0:
      continue;
    f_l_t = np.matmul(T_IMG_CAM, tmp);

    tmp = np.matmul(T_CAM_LIDAR, np.matmul(T_Lidar_Cuboid, back_right_bottom));
    if tmp[2][3] < 0:
      continue;
    b_r_b = np.matmul(T_IMG_CAM, tmp);
    tmp = np.matmul(T_CAM_LIDAR, np.matmul(T_Lidar_Cuboid, back_right_top));
    if tmp[2][3] < 0:
      continue;
    b_r_t = np.matmul(T_IMG_CAM, tmp);
    tmp = np.matmul(T_CAM_LIDAR, np.matmul(T_Lidar_Cuboid, back_left_bottom));
    if tmp[2][3] < 0:
      continue;
    b_l_b = np.matmul(T_IMG_CAM, tmp);
    tmp = np.matmul(T_CAM_LIDAR, np.matmul(T_Lidar_Cuboid, back_left_top));
    if tmp[2][3] < 0:
      continue;
    b_l_t = np.matmul(T_IMG_CAM, tmp);

    # Make sure the
    # Remove z
    f_r_b_coord = (int(f_r_b[0][3]/f_r_b[2][3]), int(f_r_b[1][3]/f_r_b[2][3]));
    f_r_t_coord = (int(f_r_t[0][3]/f_r_t[2][3]), int(f_r_t[1][3]/f_r_t[2][3]));
    f_l_b_coord = (int(f_l_b[0][3]/f_l_b[2][3]), int(f_l_b[1][3]/f_l_b[2][3]));
    f_l_t_coord = (int(f_l_t[0][3]/f_l_t[2][3]), int(f_l_t[1][3]/f_l_t[2][3]));
    if f_r_b_coord[0] < 0 or f_r_b_coord[0] > img_w or f_r_b_coord[1] < 0 or f_r_b_coord[1] > img_h:
      continue;
    if f_r_t_coord[0] < 0 or f_r_t_coord[0] > img_w or f_r_t_coord[1] < 0 or f_r_t_coord[1] > img_h:
      continue;
    if f_l_b_coord[0] < 0 or f_l_b_coord[0] > img_w or f_l_b_coord[1] < 0 or f_l_b_coord[1] > img_h:
      continue;
    if f_l_t_coord[0] < 0 or f_l_t_coord[0] > img_w or f_l_t_coord[1] < 0 or f_l_t_coord[1] > img_h:
      continue;

    b_r_b_coord = (int(b_r_b[0][3]/b_r_b[2][3]), int(b_r_b[1][3]/b_r_b[2][3]));
    b_r_t_coord = (int(b_r_t[0][3]/b_r_t[2][3]), int(b_r_t[1][3]/b_r_t[2][3]));
    b_l_b_coord = (int(b_l_b[0][3]/b_l_b[2][3]), int(b_l_b[1][3]/b_l_b[2][3]));
    b_l_t_coord = (int(b_l_t[0][3]/b_l_t[2][3]), int(b_l_t[1][3]/b_l_t[2][3]));
    if b_r_b_coord[0] < 0 or b_r_b_coord[0] > img_w or b_r_b_coord[1] < 0 or b_r_b_coord[1] > img_h:
      continue;
    if b_r_t_coord[0] < 0 or b_r_t_coord[0] > img_w or b_r_t_coord[1] < 0 or b_r_t_coord[1] > img_h:
      continue;
    if b_l_b_coord[0] < 0 or b_l_b_coord[0] > img_w or b_l_b_coord[1] < 0 or b_l_b_coord[1] > img_h:
      continue;
    if b_l_t_coord[0] < 0 or b_l_t_coord[0] > img_w or b_l_t_coord[1] < 0 or b_l_t_coord[1] > img_h:
      continue;

    # Draw  12 lines
    # Front
    cv2.line(img, f_r_b_coord, f_r_t_coord, [0, 0, 255], thickness=2, lineType=8, shift=0);
    cv2.line(img, f_r_b_coord, f_l_b_coord, [0, 0, 255], thickness=2, lineType=8, shift=0);
    cv2.line(img, f_l_b_coord, f_l_t_coord, [0, 0, 255], thickness=2, lineType=8, shift=0);
    cv2.line(img, f_l_t_coord, f_r_t_coord, [0, 0, 255], thickness=2, lineType=8, shift=0);
    # back
    cv2.line(img, b_r_b_coord, b_r_t_coord, [0, 0, 100], thickness=2, lineType=8, shift=0);
    cv2.line(img, b_r_b_coord, b_l_b_coord, [0, 0, 100], thickness=2, lineType=8, shift=0);
    cv2.line(img, b_l_b_coord, b_l_t_coord, [0, 0, 100], thickness=2, lineType=8, shift=0);
    cv2.line(img, b_l_t_coord, b_r_t_coord, [0, 0, 100], thickness=2, lineType=8, shift=0);
    # connect front to back
    cv2.line(img, f_r_b_coord, b_r_b_coord, [0, 0, 150], thickness=2, lineType=8, shift=0);
    cv2.line(img, f_r_t_coord, b_r_t_coord, [0, 0, 150], thickness=2, lineType=8, shift=0);
    cv2.line(img, f_l_b_coord, b_l_b_coord, [0, 0, 150], thickness=2, lineType=8, shift=0);
    cv2.line(img, f_l_t_coord, b_l_t_coord, [0, 0, 150], thickness=2, lineType=8, shift=0);
    #bunch of print statements I commented out lol
    #print(cuboid)
    #print(f_r_b_coord)
    #print(f_r_t_coord)
    #print(f_l_b_coord)
    #print(f_l_t_coord)

    #print(b_r_b_coord)
    #print(b_r_t_coord)
    #print(b_l_b_coord)
    #print(b_l_t_coord)

    #break;

  cv2_imshow(img)
  # cv2.imwrite("test.png", img)
  cv2.waitKey(10000)
run_demo_tracklets()

In [ ]:
def dror_filter(input_cloud):
    radius_multiplier_ = 3
    azimuth_angle_ = 0.16
    min_neighbors_ = 3
    k_neighbors_ = min_neighbors_ + 1
    min_search_radius_ = 0.04

    filtered_cloud_list = []

    # Convert PyntCloud to NumPy array
    points = input_cloud.points[['x', 'y', 'z']].to_numpy()

    # Create KDTree for neighbor search
    kd_tree = KDTree(points)

    # Go over all the points and check which don't have enough neighbors
    for p_id in range(points.shape[0]):
        x_i = points[p_id, 0]
        y_i = points[p_id, 1]
        range_i = math.sqrt(x_i**2 + y_i**2)
        search_radius_dynamic = radius_multiplier_ * azimuth_angle_ * math.pi / 180 * range_i

        if search_radius_dynamic < min_search_radius_:
            search_radius_dynamic = min_search_radius_

        distances, indices = kd_tree.query_radius([points[p_id]], r=search_radius_dynamic, return_distance=True)
        neighbors = len(indices[0]) - 1  # Subtract 1 to exclude the point itself

        # This point is not snow, add it to the filtered_cloud_list
        if neighbors >= min_neighbors_:
            filtered_cloud_list.append(points[p_id])

    filtered_cloud = np.array(filtered_cloud_list, dtype=np.float32)
    return PyntCloud(pd.DataFrame(filtered_cloud, columns=['x', 'y', 'z']))

def crop_cloud(input_cloud):
    min_vals = np.array([-4, -4, -3])
    max_vals = np.array([4, 4, 10])

    # Filter points within the bounding box
    mask = (
        (input_cloud.points['x'] >= min_vals[0]) & (input_cloud.points['x'] <= max_vals[0]) &
        (input_cloud.points['y'] >= min_vals[1]) & (input_cloud.points['y'] <= max_vals[1]) &
        (input_cloud.points['z'] >= min_vals[2]) & (input_cloud.points['z'] <= max_vals[2])
    )

    cropped_points = input_cloud.points[mask]
    return PyntCloud(cropped_points)

def print_snow_points(frame):
    seq = format(frame, '04')

    annotations_file = BASE + seq + "/3d_ann.json"
    export_annotations_file = BASE + seq + "/3d_ann_p.json"
    path_type = "labeled"

    # Load lidar data for this frame
    lidar_path = BASE + seq + "/" + path_type + "/lidar_points/data/" + format(0, '010') + ".bin"
    scan_data = np.fromfile(lidar_path, dtype=np.float32)
    lidar = scan_data.reshape((-1, 4))

    # Convert lidar 2D array to PyntCloud
    point_cloud = PyntCloud(pd.DataFrame(lidar[:, 0:3], columns=['x', 'y', 'z']))

    # Crop the point cloud
    cropped_cloud = crop_cloud(point_cloud)

    # Run DROR filter
    filtered_cloud = dror_filter(cropped_cloud)

    # Print number of snow points
    number_snow_points = len(cropped_cloud.points) - len(filtered_cloud.points)
    print(number_snow_points)

    # Visualization (PyntCloud visualization support is limited, might need to use an external tool like Open3D)
    # Example using Open3D
    import open3d as o3d

    cropped_cloud_o3d = o3d.geometry.PointCloud()
    cropped_cloud_o3d.points = o3d.utility.Vector3dVector(cropped_cloud.points[['x', 'y', 'z']].to_numpy())

    o3d.visualization.draw_plotly([cropped_cloud_o3d])

BASE = '/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/'
LOOP = False

print_snow_points(1) #sensor 1 of 3/6/18 data

In [ ]:
def bev(s1,s2,f1,f2,frame,cam):
    '''

    :param s1: example 15 (15 meter to the left of the car)
    :param s2: s2 meters from the right of the car
    :param f1: f1 meters from the front of the car
    :param f2: f2 meters from the back of the car
    :param frame: the frame number
    :return:
    '''
    camNum = '0001/'
    DISTORTED = False

    if DISTORTED:
        path_type = 'raw'
    else:
        path_type = 'processedmoose'
    path_type = 'labeled' #according to my own file structure

    #lidar_path = "/media/wavelab/d3cd89ab-7705-4996-94f3-01da25ba8f50/autonomoose/" + path_type + "/lidar_points/data/" + format(frame, '010') + ".bin"
    lidar_path = "/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/" + camNum + path_type + "/lidar_points/data/" + format(frame, '010') + ".bin"
    #limit the viewing range
    side_range = [-s1,s2] #15 meters from either side of the car
    fwd_range = [-f1,f2] # 15 m infront of the car

    scan_data = np.fromfile(lidar_path, dtype= np.float32) #numpy from file reads binary file
    #scan_data is a single row of all the lidar values
    # 2D array where each row contains a point [x, y, z, intensity]
    #we covert scan_data to format said above
    lidar = scan_data.reshape((-1, 4));

    lidar_x = lidar[:,0]
    lidar_y = lidar[:,1]
    lidar_z = lidar [:,2]



    lidar_x_trunc = []
    lidar_y_trunc = []
    lidar_z_trunc = []

    for i in range(len(lidar_x)):
        if lidar_x[i] > fwd_range[0] and lidar_x[i] < fwd_range[1]: #get the lidar coordinates
            if lidar_y[i] > side_range[0] and lidar_y[i] < side_range[1]:

                lidar_x_trunc.append(lidar_x[i])
                lidar_y_trunc.append(lidar_y[i])
                lidar_z_trunc.append(lidar_z[i])

    # to use for the plot
    x_img = [i* -1 for i in lidar_y_trunc] #in the image plot, the negative lidar y axis is the img x axis
    y_img = lidar_x_trunc #the lidar x axis is the img y axis
    pixel_values = lidar_z_trunc


    #shift values such that 0,0 is the minimum
    x_img = [i -side_range[0] for i in x_img]
    y_img = [i -fwd_range[0] for i in y_img]




    '''
    tracklets
    '''


    #annotations_file = '/media/wavelab/d3cd89ab-7705-4996-94f3-01da25ba8f50/autonomoose/3d_annotations.json';
    annotations_file = '/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/' + camNum + '3d_ann.json'
    # Load 3d annotations
    annotations_data = None
    with open(annotations_file) as f:
        annotations_data = json.load(f)

    # Add each cuboid to image
    '''
    Rotations in 3 dimensions can be represented by a sequece of 3 rotations around a sequence of axes.
    In theory, any three axes spanning the 3D Euclidean space are enough. In practice the axes of rotation are chosen to be the basis vectors.
    The three rotations can either be in a global frame of reference (extrinsic) or in a body centred frame of refernce (intrinsic),
    which is attached to, and moves with, the object under rotation
    '''


    # PLOT THE IMAGE
    cmap = "jet"    # Color map to use
    dpi = 100       # Image resolution
    x_max = side_range[1] - side_range[0]
    y_max = fwd_range[1] - fwd_range[0]
    fig, ax = plt.subplots(figsize=(1000/dpi, 1000/dpi), dpi=dpi)

    # the coordinates in the tracklet json are lidar coords
    x_trunc = []
    y_trunc = []
    x_1 = []
    x_2 =[]
    x_3 = []
    x_4 =[]
    y_1 =[]
    y_2 =[]
    y_3 = []
    y_4 =[]

    for cuboid in annotations_data[frame]['cuboids']:
        T_Lidar_Cuboid = np.eye(4);  # identify matrix
        T_Lidar_Cuboid[0:3, 0:3] = R.from_euler('z', cuboid['yaw'],
                                                degrees=False).as_matrix();  # rotate the identity matrix #changed as_dcm to as_matrix
        T_Lidar_Cuboid[0][3] = cuboid['position']['x'];  # center of the tracklet, from cuboid to lidar
        T_Lidar_Cuboid[1][3] = cuboid['position']['y'];
        T_Lidar_Cuboid[2][3] = cuboid['position']['z'];



        if cuboid['position']['x']> fwd_range[0] and cuboid['position']['x'] < fwd_range[1]: #make sure the cuboid is within the range we want to see
            if cuboid['position']['y'] > side_range[0] and cuboid['position']['y'] < side_range[1]:
                x_trunc.append(cuboid['position']['x'])
                y_trunc.append(cuboid['position']['y'])

                width = cuboid['dimensions']['x'];
                length = cuboid['dimensions']['y'];
                height = cuboid['dimensions']['z'];
                radius = 3


                #the top view of the tracklet in the "cuboid frame". The cuboid frame is a cuboid with origin (0,0,0)
                #we are making a cuboid that has the dimensions of the tracklet but is located at the origin
                front_right_top = np.array(
                    [[1, 0, 0, length / 2], [0, 1, 0, width / 2], [0, 0, 1, height / 2], [0, 0, 0, 1]]);

                front_left_top = np.array(
                    [[1, 0, 0, length / 2], [0, 1, 0, -width / 2], [0, 0, 1, height / 2], [0, 0, 0, 1]]);


                back_right_top = np.array(
                    [[1, 0, 0, -length / 2], [0, 1, 0, width / 2], [0, 0, 1, height / 2], [0, 0, 0, 1]]);

                back_left_top = np.array(
                    [[1, 0, 0, -length / 2], [0, 1, 0, -width / 2], [0, 0, 1, height / 2], [0, 0, 0, 1]]);

                # Project to lidar


                f_r_t =  np.matmul(T_Lidar_Cuboid, front_right_top)
                f_l_t  = np.matmul(T_Lidar_Cuboid, front_left_top)
                b_r_t  = np.matmul(T_Lidar_Cuboid, back_right_top)
                b_l_t = np.matmul(T_Lidar_Cuboid, back_left_top)


                x_1.append(f_r_t[0][3])
                y_1.append(f_r_t[1][3])

                x_2.append(f_l_t[0][3])
                y_2.append(f_l_t[1][3])

                x_3.append(b_r_t[0][3])
                y_3.append(b_r_t[1][3])

                x_4.append(b_l_t[0][3])
                y_4.append(b_l_t[1][3])






    # to use for the plot

    x_img_tracklets = [i * -1 for i in y_trunc]  # in the image to plot, the negative lidar y axis is the img x axis
    y_img_tracklets = x_trunc  # the lidar x axis is the img y axis

    x_img_1 = [i * -1 for i in y_1]
    y_img_1 = x_1

    x_img_2 = [i * -1 for i in y_2]
    y_img_2 = x_2

    x_img_3 = [i * -1 for i in y_3]
    y_img_3 = x_3

    x_img_4 = [i * -1 for i in y_4]
    y_img_4 = x_4



    # shift values such that 0,0 is the minimum
    x_img_tracklets = [i -side_range[0] for i in x_img_tracklets]
    y_img_tracklets = [i -fwd_range[0] for i in y_img_tracklets]

    x_img_1 = [i -side_range[0] for i in x_img_1]
    y_img_1 = [i - fwd_range[0] for i in y_img_1]

    x_img_2 = [i -side_range[0] for i in x_img_2]
    y_img_2 = [i - fwd_range[0] for i in y_img_2]

    x_img_3 = [i -side_range[0] for i in x_img_3]
    y_img_3 = [i - fwd_range[0] for i in y_img_3]

    x_img_4 = [i -side_range[0] for i in x_img_4]
    y_img_4 = [i - fwd_range[0] for i in y_img_4]

    for i in range(len(x_img_1)): #plot the tracklets
        poly = np.array([[x_img_1[i], y_img_1[i]], [x_img_2[i], y_img_2[i]], [x_img_4[i], y_img_4[i]], [x_img_3[i], y_img_3[i]]])
        polys = patches.Polygon(poly,closed=True,fill=False, edgecolor ='c', linewidth=1)
        ax.add_patch(polys)



    ax.scatter(x_img_tracklets,y_img_tracklets, marker ='o', color='red', linewidths=1) #center of the tracklets
    ax.scatter(x_img, y_img, s=1, c=pixel_values, alpha=1, cmap=cmap)
    ax.set_facecolor((0, 0, 0))  # backgroun is black
    ax.axis('scaled')  # {equal, scaled}
    ax.xaxis.set_visible(False)  # Do not draw axis tick marks
    ax.yaxis.set_visible(False)  # Do not draw axis tick marks
    plt.xlim([0, x_max])
    plt.ylim([0, y_max])
    #fig.savefig("/media/wavelab/d3cd89ab-7705-4996-94f3-01da25ba8f50/autonomoose/devmoose/moose_bev_" + str(frame) + ".png", dpi=dpi, bbox_inches='tight', pad_inches=0.0)
    fig.savefig("/content/drive/MyDrive/Colab Notebooks/"  + "moose_bev_" + str(frame) + ".png", dpi=dpi, bbox_inches='tight', pad_inches=0.0)

    return 0

bev(15,15,15,15,11,0)

In [ ]:
#run demo lidar bev
frame = 12
cam = '0'
seq = '0001'
DISTORTED = False
MOVE_FORWARD = True
DISPLAY_LIDAR = False
DISPLAY_CUBOID_CENTER = False
MIN_CUBOID_DIST = 40.0

#BASE = '/media/matthew/WAVELAB_2TB/winter/data/'
# BASE = '/media/matthew/MOOSE-4TB/2019_02_27/'
# BASE = '/media/matthew/MOOSE-4TB/2018_03_06/data/'
# BASE = '/media/matthew/MOOSE-4TB/2018_03_07/data/'
BASE = '/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/'



if DISTORTED:
  path_type = 'raw'
else:
  path_type = 'processed'
path_type='labeled'

lidar_path = BASE + seq + "/" + path_type + "/lidar_points/data/" + format(frame, '010') + ".bin";
#calib_path = "/media/matthew/WAVELAB_2TB/winter/calib/";
calib_path = "/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/calib/"
img_path =  BASE + seq + "/" + path_type + "/image_0" + cam + "/data/" + format(frame, '010') + ".png";
annotations_path =  BASE + seq + "/3d_ann.json";

def bev(s1,s2,f1,f2,frame,lidar_path,annotations_path):
    '''

    :param s1: example 15 (15 meter to the left of the car)
    :param s2: s2 meters from the right of the car
    :param f1: f1 meters from the front of the car
    :param f2: f2 meters from the back of the car
    :param frame: the frame number
    :return:
    '''

    #limit the viewing range
    side_range = [-s1,s2] #15 meters from either side of the car
    fwd_range = [-f1,f2] # 15 m infront of the car

    scan_data = np.fromfile(lidar_path, dtype= np.float32) #numpy from file reads binary file
    #scan_data is a single row of all the lidar values
    # 2D array where each row contains a point [x, y, z, intensity]
    #we covert scan_data to format said above
    lidar = scan_data.reshape((-1, 4));

    lidar_x = lidar[:,0]
    lidar_y = lidar[:,1]
    lidar_z = lidar [:,2]



    lidar_x_trunc = []
    lidar_y_trunc = []
    lidar_z_trunc = []

    for i in range(len(lidar_x)):
        if lidar_x[i] > fwd_range[0] and lidar_x[i] < fwd_range[1]: #get the lidar coordinates
            if lidar_y[i] > side_range[0] and lidar_y[i] < side_range[1]:

                lidar_x_trunc.append(lidar_x[i])
                lidar_y_trunc.append(lidar_y[i])
                lidar_z_trunc.append(lidar_z[i])

    # to use for the plot
    x_img = [i* -1 for i in lidar_y_trunc] #in the image plot, the negative lidar y axis is the img x axis
    y_img = lidar_x_trunc #the lidar x axis is the img y axis
    pixel_values = lidar_z_trunc


    #shift values such that 0,0 is the minimum
    x_img = [i -side_range[0] for i in x_img]
    y_img = [i -fwd_range[0] for i in y_img]




    '''
    tracklets
    '''

    # Load 3d annotations
    annotations_data = None
    with open(annotations_path) as f:
        annotations_data = json.load(f)

    # Add each cuboid to image
    '''
    Rotations in 3 dimensions can be represented by a sequece of 3 rotations around a sequence of axes.
    In theory, any three axes spanning the 3D Euclidean space are enough. In practice the axes of rotation are chosen to be the basis vectors.
    The three rotations can either be in a global frame of reference (extrinsic) or in a body centred frame of refernce (intrinsic),
    which is attached to, and moves with, the object under rotation
    '''


    # PLOT THE IMAGE
    cmap = "jet"    # Color map to use
    dpi = 100       # Image resolution
    x_max = side_range[1] - side_range[0]
    y_max = fwd_range[1] - fwd_range[0]
    fig, ax = plt.subplots(figsize=(2000/dpi, 2000/dpi), dpi=dpi)

    # the coordinates in the tracklet json are lidar coords
    x_trunc = []
    y_trunc = []
    x_1 = []
    x_2 =[]
    x_3 = []
    x_4 =[]
    y_1 =[]
    y_2 =[]
    y_3 = []
    y_4 =[]

    for cuboid in annotations_data[frame]['cuboids']:
        T_Lidar_Cuboid = np.eye(4);  # identify matrix
        T_Lidar_Cuboid[0:3, 0:3] = R.from_euler('z', cuboid['yaw'],
                                                degrees=False).as_matrix();  # rotate the identity matrix
        T_Lidar_Cuboid[0][3] = cuboid['position']['x'];  # center of the tracklet, from cuboid to lidar
        T_Lidar_Cuboid[1][3] = cuboid['position']['y'];
        T_Lidar_Cuboid[2][3] = cuboid['position']['z'];



        if cuboid['position']['x']> fwd_range[0] and cuboid['position']['x'] < fwd_range[1]: #make sure the cuboid is within the range we want to see
            if cuboid['position']['y'] > side_range[0] and cuboid['position']['y'] < side_range[1]:
                x_trunc.append(cuboid['position']['x'])
                y_trunc.append(cuboid['position']['y'])

                width = cuboid['dimensions']['x'];
                length = cuboid['dimensions']['y'];
                height = cuboid['dimensions']['z'];
                radius = 3


                #the top view of the tracklet in the "cuboid frame". The cuboid frame is a cuboid with origin (0,0,0)
                #we are making a cuboid that has the dimensions of the tracklet but is located at the origin
                front_right_top = np.array(
                    [[1, 0, 0, length / 2], [0, 1, 0, width / 2], [0, 0, 1, height / 2], [0, 0, 0, 1]]);

                front_left_top = np.array(
                    [[1, 0, 0, length / 2], [0, 1, 0, -width / 2], [0, 0, 1, height / 2], [0, 0, 0, 1]]);


                back_right_top = np.array(
                    [[1, 0, 0, -length / 2], [0, 1, 0, width / 2], [0, 0, 1, height / 2], [0, 0, 0, 1]]);

                back_left_top = np.array(
                    [[1, 0, 0, -length / 2], [0, 1, 0, -width / 2], [0, 0, 1, height / 2], [0, 0, 0, 1]]);

                # Project to lidar


                f_r_t =  np.matmul(T_Lidar_Cuboid, front_right_top)
                f_l_t  = np.matmul(T_Lidar_Cuboid, front_left_top)
                b_r_t  = np.matmul(T_Lidar_Cuboid, back_right_top)
                b_l_t = np.matmul(T_Lidar_Cuboid, back_left_top)


                x_1.append(f_r_t[0][3])
                y_1.append(f_r_t[1][3])

                x_2.append(f_l_t[0][3])
                y_2.append(f_l_t[1][3])

                x_3.append(b_r_t[0][3])
                y_3.append(b_r_t[1][3])

                x_4.append(b_l_t[0][3])
                y_4.append(b_l_t[1][3])






    # to use for the plot

    x_img_tracklets = [i * -1 for i in y_trunc]  # in the image to plot, the negative lidar y axis is the img x axis
    y_img_tracklets = x_trunc  # the lidar x axis is the img y axis

    x_img_1 = [i * -1 for i in y_1]
    y_img_1 = x_1

    x_img_2 = [i * -1 for i in y_2]
    y_img_2 = x_2

    x_img_3 = [i * -1 for i in y_3]
    y_img_3 = x_3

    x_img_4 = [i * -1 for i in y_4]
    y_img_4 = x_4



    # shift values such that 0,0 is the minimum
    x_img_tracklets = [i -side_range[0] for i in x_img_tracklets]
    y_img_tracklets = [i -fwd_range[0] for i in y_img_tracklets]

    x_img_1 = [i -side_range[0] for i in x_img_1]
    y_img_1 = [i - fwd_range[0] for i in y_img_1]

    x_img_2 = [i -side_range[0] for i in x_img_2]
    y_img_2 = [i - fwd_range[0] for i in y_img_2]

    x_img_3 = [i -side_range[0] for i in x_img_3]
    y_img_3 = [i - fwd_range[0] for i in y_img_3]

    x_img_4 = [i -side_range[0] for i in x_img_4]
    y_img_4 = [i - fwd_range[0] for i in y_img_4]

    for i in range(len(x_img_1)): #plot the tracklets
        poly = np.array([[x_img_1[i], y_img_1[i]], [x_img_2[i], y_img_2[i]], [x_img_4[i], y_img_4[i]], [x_img_3[i], y_img_3[i]]])
        polys = patches.Polygon(poly,closed=True,fill=False, edgecolor ='r', linewidth=1)
        ax.add_patch(polys)



    # ax.scatter(x_img_tracklets,y_img_tracklets, marker ='o', color='red', linewidths=1) #center of the tracklets
    ax.scatter(x_img, y_img, s=1, c=pixel_values, alpha=1.0, cmap=cmap)
    ax.set_facecolor((0, 0, 0))  # backgrounD is black
    ax.axis('scaled')  # {equal, scaled}
    ax.xaxis.set_visible(False)  # Do not draw axis tick marks
    ax.yaxis.set_visible(False)  # Do not draw axis tick marks
    plt.xlim([0, x_max])
    plt.ylim([0, y_max])
    #fig.savefig("/home/matthew/Desktop/bev_" + str(frame) + ".png", dpi=dpi, bbox_inches='tight', pad_inches=0.0)
    fig.savefig("/content/drive/MyDrive/Colab Notebooks" + str(frame) + ".png", dpi=dpi, bbox_inches='tight', pad_inches=0.0)

bev(50,50,50,50,frame,lidar_path,annotations_path)

In [ ]:
def demo_lidar():
  frame = 00
  cam = '0'
  #seq = '0027'
  seq = '0001'
  DISTORTED = False
  MOVE_FORWARD = True
  #BASE = "/media/matthew/WAVELAB_2TB/winter/"
  BASE = "/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/"

  if DISTORTED:
    path_type = 'raw'
  else:
    path_type = 'processed'
  path_type = 'labeled'


  lidar_path = BASE  + seq + "/" + path_type + "/lidar_points/" + "data/" +  format(frame, '010') + ".bin";
  calib_path = BASE + "calib/";
  img_path = BASE + seq + "/" + path_type + "/image_0" + cam + "/data/"  + format(frame, '010') + ".png";
  print("lidar_path:" + lidar_path)
  print("img_path:" + img_path)

  # load calibration dictionary
  calib = load_calibration(calib_path);

  # Projection matrix from camera to image frame
  T_IMG_CAM = np.eye(4);
  T_IMG_CAM[0:3,0:3] = np.array(calib['CAM0' + cam]['camera_matrix']['data']).reshape(-1, 3);
  T_IMG_CAM = T_IMG_CAM[0:3,0:4]; # remove last row

  T_CAM_LIDAR = np.linalg.inv(np.array(calib['extrinsics']['T_LIDAR_CAM0' + cam]));

  dist_coeffs = np.array(calib['CAM0' + cam]['distortion_coefficients']['data'])

  lidar_utils_obj = lidar_utils(T_CAM_LIDAR);

  while True:
    print(frame)
    # read image
    img = cv2.imread(img_path)

    # Project points onto image
    img = lidar_utils_obj.project_points(img, lidar_path, T_IMG_CAM, T_CAM_LIDAR, dist_coeffs, DISTORTED);
    # cv2.imwrite("test.png", img)

    cv2_imshow(img)
    cv2.waitKey(1000)

    if MOVE_FORWARD:
      frame += 1;
      lidar_path = BASE  + seq + "/" + path_type + "/lidar_points/" + "data/" +  format(frame, '010') + ".bin";

      img_path = BASE + seq + "/" + path_type + "/image_0" + cam + "/data/"  + format(frame, '010') + ".png";
      img = cv2.imread(img_path)
#testing lol
demo_lidar()

In [ ]:
def run_demo_2d_tracklets():
  frame = 0
  cam = '0'
  #seq = '0069'
  seq = '0018'
  DISTORTED = False
  MOVE_FORWARD = True
  #base_path = "/media/matthew/WAVELAB_2TB/winter"
  #calib_path = "/media/matthew/WAVELAB_2TB/winter/calib/";
  base_path = "/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/"
  calib_path = "/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/calib/"

  if DISTORTED:
    path_type = 'raw'
  else:
    path_type = 'processed'

  img_path = base_path + "/data/" + seq + "/" + path_type + "/image_0" + cam + "/data/" + format(frame, '010') + ".png";
  annotations_file = base_path + "/data/" + seq + "/" + '/2d_annotations.json';
  #annotations_file = base_path + seq + '/3d_ann.json'; #changed according to my folder

  # load calibration dictionary
  calib = load_calibration(calib_path);

  # Projection matrix from camera to image frame
  T_IMG_CAM = np.eye(4);
  T_IMG_CAM[0:3,0:3] = np.array(calib['CAM0' + cam]['camera_matrix']['data']).reshape(-1, 3);
  T_IMG_CAM = T_IMG_CAM[0:3,0:4]; # remove last row

  dist_coeffs = np.array(calib['CAM0' + cam]['distortion_coefficients']['data'])

  # Load 2d annotations
  annotations_data = None
  with open(annotations_file) as f:
    annotations_data = json.load(f)

  img = cv2.imread(img_path)
  img_h, img_w = img.shape[:2]

  # Add each box to image
  for camera_response in annotations_data[frame]['camera_responses']:
    if camera_response['camera_used'] != int(cam):
      continue;

    for annotation in camera_response['annotations']:
      left = int(annotation['left'])
      top = int(annotation['top'])
      width = int(annotation['width'])
      height = int(annotation['height'])

      if DISTORTED:
        cv2.rectangle(img,(left,top),(left + width,top + height),(0,255,0),thickness=3)
      else:
        pts_uv = np.array([[[left,top]],[[left + width,top + height]]], dtype=np.float32)
        new_pts = cv2.undistortPoints(pts_uv, T_IMG_CAM[0:3,0:3], dist_coeffs, P=T_IMG_CAM[0:3,0:3])
        cv2.rectangle(img,(new_pts[0][0][0],new_pts[0][0][1]),(new_pts[1][0][0],new_pts[1][0][1]),(0,255,0),thickness=3)

  cv2.imshow('image',img)
  cv2.waitKey(10000)
run_demo_2d_tracklets()

#Install/Imports & Files

In [ ]:
#installations
!git clone https://github.com/ruhyadi/YOLO3D.git
!pip install open3d
!pip install ultralytics
!pip install -r /content/YOLO3D/requirements.txt
!pip install roboflow
!pip install Ipython
!pip install comet-ml




In [ ]:
#imports
import comet_ml
import tensorflow as tf
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
import sklearn as sk
import cv2 as cv
import json
import ultralytics
import sys
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from roboflow import Roboflow
import yaml
from PIL import Image
from IPython.core.magic import register_line_cell_magic

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Paths to images and data
BASE = "/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/"
seq = '0010'
path_type = 'labeled'
frame = 26
cam = '0'


CALIB_BASE = "/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/"
lidar_path = BASE + seq + "/" + path_type + "/lidar_points/data/" + format(frame, '010') + ".bin";
calib_path = CALIB_BASE + "calib";
img_path = BASE + seq + "/" + path_type + "/image_0" + cam + "/data/" + format(frame, '010') + ".png";

#YoloV5 Model (Working)

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
#X_train = cv2.imread('/content/drive/MyDrive/Data Repo/cadcd/2018_03_06/0001/labeled/image_00/data', cv2.IMREAD_ANYCOLOR)


from roboflow import Roboflow
rf = Roboflow(api_key="q6p45y5b9vFNSaPd8IT1")
project = rf.workspace("yolodetectionav").project("yoloav")
version = project.version(1)
dataset = version.download("yolov5")



In [ ]:
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
print(num_classes)

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))


In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 237 --batch 16 --epochs 300 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs




In [ ]:




from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/val_batch0_labels.jpg', width=900)

In [ ]:
# print out confusion matrix
Image(filename = "/content/yolov5/runs/train/yolov5s_results/confusion_matrix.png", width = 800)


In [ ]:
results = pd.read_csv("/content/yolov5/runs/train/yolov5s_results/results.csv")
#graph all losses over the epochs



In [ ]:
run_demo_tracklets()

In [ ]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data={dataset.location}/data.yaml \
epochs=100 \
imgsz=640

#YoloV9 Model (Non-Working)

In [ ]:
#pretrained YOLOv9 Model
model = YOLO('yolov9c.pt')
results = model(img_path, show = True)


cv2_imshow(results[0].plot())
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
#loading in the yolo v9 repo
# clone YOLOv5 repository
!git clone https://github.com/WongKinYiu/yolov9  # clone repo
%cd yolov9
#!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99 #this was in the yolo v5 code
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
#using roboflow customized dataset
from roboflow import Roboflow
rf = Roboflow(api_key="q6p45y5b9vFNSaPd8IT1")
project = rf.workspace("yolodetectionav").project("yoloav")
version = project.version(1)
dataset = version.download("yolov9")


In [ ]:
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
print(num_classes)

#with open("/content/yolov9/data/data.yaml", 'r') as stream:
    #num_classes = str(yaml.safe_load(stream)['nc'])
#print(num_classes)
print(dataset.location)

In [ ]:
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov9/models/custom_yolov9s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0
width_multiple: 1.0

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv9 backbone
backbone:
  [
   [-1, 1, Silence, []],

   # conv down
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2

   # conv down
   [-1, 1, Conv, [128, 3, 2]],  # 2-P2/4

   # elan-1 block
   [-1, 1, RepNCSPELAN4, [256, 128, 64, 1]],  # 3

   # conv down
   [-1, 1, Conv, [256, 3, 2]],  # 4-P3/8

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 256, 128, 1]],  # 5

   # conv down
   [-1, 1, Conv, [512, 3, 2]],  # 6-P4/16

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 7

   # conv down
   [-1, 1, Conv, [512, 3, 2]],  # 8-P5/32

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 9
  ]

# YOLOv9 head
head:
  [
   # elan-spp block
   [-1, 1, SPPELAN, [512, 256]],  # 10

   # up-concat merge
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 7], 1, Concat, [1]],  # cat backbone P4

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 13

   # up-concat merge
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 5], 1, Concat, [1]],  # cat backbone P3

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [256, 256, 128, 1]],  # 16 (P3/8-small)

   # conv-down merge
   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 13], 1, Concat, [1]],  # cat head P4

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 19 (P4/16-medium)

   # conv-down merge
   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 22 (P5/32-large)

   # routing
   [5, 1, CBLinear, [[256]]], # 23
   [7, 1, CBLinear, [[256, 512]]], # 24
   [9, 1, CBLinear, [[256, 512, 512]]], # 25

   # conv down
   [0, 1, Conv, [64, 3, 2]],  # 26-P1/2

   # conv down
   [-1, 1, Conv, [128, 3, 2]],  # 27-P2/4

   # elan-1 block
   [-1, 1, RepNCSPELAN4, [256, 128, 64, 1]],  # 28

   # conv down fuse
   [-1, 1, Conv, [256, 3, 2]],  # 29-P3/8
   [[23, 24, 25, -1], 1, CBFuse, [[0, 0, 0]]], # 30

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 256, 128, 1]],  # 31

   # conv down fuse
   [-1, 1, Conv, [512, 3, 2]],  # 32-P4/16
   [[24, 25, -1], 1, CBFuse, [[1, 1]]], # 33

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 34

   # conv down fuse
   [-1, 1, Conv, [512, 3, 2]],  # 35-P5/32
   [[25, -1], 1, CBFuse, [[2]]], # 36

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 37

   # detect
   [[31, 34, 37, 16, 19, 22], 1, DualDDetect, [nc]],  # DualDDetect(A3, A4, A5, P3, P4, P5)
  ]

In [ ]:
!pip uninstall comet
!pip install comet
!pip install pyfocus

In [ ]:
import os
os.environ['COMET_OFFLINE_DIRECTORY'] = '/content/yolov9'

In [ ]:
%%time
#%cd yolov9
!python train.py --img 207 --batch 16 --epochs 100 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov9s.yaml --weights '' --name yolov9s_results  --cache --hyp ./data/hyps/hyp.scratch-high.yaml





#YOLO 3D